In [1]:
from pathlib import Path

import cv2 as cv
import numpy as np

### **1. Camera Calibration**

In [2]:
def create_world_grid(pattern_shape, pattern_size_mm):
    world_grid = np.indices((pattern_shape[0], pattern_shape[1])).transpose().reshape(-1, 2)
    world_grid = np.hstack((world_grid, np.zeros((world_grid.shape[0], 1))))
    world_grid = np.multiply(world_grid, pattern_size_mm).astype(np.float32)
    return world_grid

In [3]:
DEVICE = "thinkpad"
IMAGES_DIR = Path(f"./assets/calib/{DEVICE}/")
PATTERN_SHAPE = (10, 7)  # Inner corners.
PATTERN_SIZE_MM = 25  # Size of the squares in mm.

CORNER_SUB_PIX_CRITERIA = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
CHESSBOARD_CORNERS_FLAGS = cv.CALIB_CB_ADAPTIVE_THRESH | cv.CALIB_CB_NORMALIZE_IMAGE

world_grid = create_world_grid(PATTERN_SHAPE, PATTERN_SIZE_MM)

In [4]:
image_points = list()
world_points = list()

for image_path in IMAGES_DIR.glob("*.jpg"):
    image = cv.imread(str(image_path))
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    found, corners = cv.findChessboardCorners(image_gray, PATTERN_SHAPE, None, CHESSBOARD_CORNERS_FLAGS)
    print(f"Found corners in {image_path.name}: {found}")

    if found:
        corners = cv.cornerSubPix(image_gray, corners, (11, 11), (-1, -1), CORNER_SUB_PIX_CRITERIA)
        image_points.append(corners)
        world_points.append(world_grid)

    cv.drawChessboardCorners(image, PATTERN_SHAPE, corners, found)
    cv.imshow("Chessboard", image)
    cv.waitKey(1000)

cv.destroyAllWindows()

rms, camera_matrix, dist_coefs, rvecs, tvecs = cv.calibrateCamera(
    objectPoints=world_points,
    imagePoints=image_points,
    imageSize=image_gray.shape[::-1],
    cameraMatrix=None,  # type: ignore
    distCoeffs=None,  # type: ignore
    flags=cv.CALIB_FIX_K3 | cv.CALIB_FIX_TANGENT_DIST,
)

np.savetxt(f"./assets/{DEVICE}_camera_matrix.txt", camera_matrix, fmt="%f")
np.savetxt(f"./assets/{DEVICE}_dist_coeffs.txt", dist_coefs, fmt="%f")

print(f"RMS: {rms}")
print(f"Camera Matrix:\n {camera_matrix}")
print(f"Distortion Coefficients:\n {dist_coefs}")

Found corners in frame_0180.jpg: True
Found corners in frame_0280.jpg: True
Found corners in frame_0375.jpg: True
Found corners in frame_0510.jpg: True
Found corners in frame_0614.jpg: True
Found corners in frame_0750.jpg: True
Found corners in frame_0910.jpg: True
Found corners in frame_1094.jpg: True
Found corners in frame_1200.jpg: True
Found corners in frame_1312.jpg: True
Found corners in frame_1605.jpg: True
Found corners in frame_1764.jpg: True
Found corners in frame_1853.jpg: True
RMS: 0.780975885360315
Camera Matrix:
 [[981.07179941   0.         635.39524088]
 [  0.         984.28346233 350.49213589]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[-0.05406293  0.01452548  0.          0.          0.        ]]


### **2. Camera Pose Tracking**

In [6]:
cap = cv.VideoCapture(0)
camera_matrix = np.loadtxt("./assets/predator_camera_matrix.txt")
dist_coefs = np.loadtxt("./assets/predator_dist_coeffs.txt")

cube_size = 2 * PATTERN_SIZE_MM
cube3D = np.array(
    [
        [0, 0, 0],  # Corner 1 (origin)
        [cube_size, 0, 0],  # Corner 2
        [cube_size, cube_size, 0],  # Corner 3
        [0, cube_size, 0],  # Corner 4
        [0, 0, -cube_size],  # Corner 5
        [cube_size, 0, -cube_size],  # Corner 6
        [cube_size, cube_size, -cube_size],  # Corner 7
        [0, cube_size, -cube_size],  # Corner 8
    ],
    dtype=np.float32,
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    found, corners = cv.findChessboardCorners(image, PATTERN_SHAPE, None, CHESSBOARD_CORNERS_FLAGS)

    if found:
        corners = cv.cornerSubPix(image, corners, (11, 11), (-1, -1), CORNER_SUB_PIX_CRITERIA)
        _, rvecs, tvecs = cv.solvePnP(
            world_grid,
            corners,
            camera_matrix,
            dist_coefs,
            useExtrinsicGuess=False,
            flags=cv.SOLVEPNP_IPPE,
        )

        cube2D, _ = cv.projectPoints(cube3D, rvecs, tvecs, camera_matrix, dist_coefs)
        cube2D = cube2D.astype(np.intp).reshape(-1, 2)

        cv.drawContours(frame, [cube2D[:4]], -1, (0, 0, 255), 2, cv.LINE_AA)  # Bottom face
        cv.drawContours(frame, [cube2D[4:]], -1, (0, 255, 0), 2, cv.LINE_AA)  # Top face
        for i in range(4):  # Sides.
            cv.line(frame, tuple(cube2D[i]), tuple(cube2D[i + 4]), (255, 0, 0), 2, cv.LINE_AA)

        # cv.drawFrameAxes(frame, camera_matrix, dist_coefs, rvecs, tvecs, PATTERN_SIZE_MM)

    frame = cv.undistort(frame, camera_matrix, dist_coefs)

    cv.imshow("Chessboard", frame)
    if cv.waitKey(1) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()